In [17]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset_20' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'HR_data')
output_dir = os.path.join(home, 'video_resources', 'sql_database')

In [18]:
from sql_face.alchemy import get_session
session = get_session(output_dir, db_name)

In [19]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sql_face.tables import  QualityImage, FaceImage, CroppedImage, Image, QualityModel, QualityGroup,Detector, EmbeddingModel,EnfsiImage,EnfsiVideoFrame

Ficheros a importar

In [20]:
database = 'SCFace'

emb_file = os.path.join(input_dir,f'embeddings_{database}_hr.npy')
filenames_file = os.path.join(input_dir,f'filenames_{database}_hr.npy')
quality_file = os.path.join(input_dir,f'results_{database}_hr.csv')

emb = np.load(emb_file) 
filename = np.load(filenames_file)
df_quality = pd.read_csv(quality_file)


In [21]:
df_quality

,Path,Quality
0,/content/CodeFormer/align_hr_UTKface/91557.png,0.796164
1,/content/CodeFormer/align_hr_UTKface/91116.png,0.746011
2,/content/CodeFormer/align_hr_UTKface/70556.png,0.719257
3,/content/CodeFormer/align_hr_UTKface/72920.png,0.194310
4,/content/CodeFormer/align_hr_UTKface/70545.png,0.516199
...,...,...
23950,/content/CodeFormer/align_hr_UTKface/86601.png,0.175016
23951,/content/CodeFormer/align_hr_UTKface/87145.png,0.502189
23952,/content/CodeFormer/align_hr_UTKface/89715.png,0.903959
23953,/content/CodeFormer/align_hr_UTKface/89977.png,0.489398


In [22]:
print(filename.shape,emb.shape, df_quality.shape)


(23955,) (23955, 512) (23955, 2)


In [23]:
df = pd.DataFrame({'Path': filename, 'embedding': emb.tolist()})
df

,Path,embedding
0,/content/CodeFormer/align_hr_UTKface/91557.png,"[-1.210444688796997, -0.4962098002433777, 1.04..."
1,/content/CodeFormer/align_hr_UTKface/91116.png,"[0.9080809354782104, -0.9733660221099854, -1.0..."
2,/content/CodeFormer/align_hr_UTKface/70556.png,"[-0.6530567407608032, -0.26220884919166565, 1...."
3,/content/CodeFormer/align_hr_UTKface/72920.png,"[-2.296236753463745, 0.2565675973892212, 0.116..."
4,/content/CodeFormer/align_hr_UTKface/70545.png,"[-0.7272638082504272, 1.2198829650878906, 0.95..."
...,...,...
23950,/content/CodeFormer/align_hr_UTKface/86601.png,"[0.2633347511291504, 0.4482688009738922, 1.175..."
23951,/content/CodeFormer/align_hr_UTKface/87145.png,"[-2.198603630065918, 0.04760923981666565, 1.21..."
23952,/content/CodeFormer/align_hr_UTKface/89715.png,"[-0.8513012528419495, 0.602628767490387, 0.446..."
23953,/content/CodeFormer/align_hr_UTKface/89977.png,"[-3.269528388977051, 0.27106744050979614, 2.29..."


In [24]:
df['CI_id'] = df.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))
df_quality['CI_id'] = df_quality.Path.apply(lambda x: int(os.path.basename(x).split(".")[0]))

In [25]:
results_data = df.merge(df_quality, on='CI_id')

In [26]:
results_data.shape

(23955, 5)

In [27]:
results_data

,Path_x,embedding,CI_id,Path_y,Quality
0,/content/CodeFormer/align_hr_UTKface/91557.png,"[-1.210444688796997, -0.4962098002433777, 1.04...",91557,/content/CodeFormer/align_hr_UTKface/91557.png,0.796164
1,/content/CodeFormer/align_hr_UTKface/91116.png,"[0.9080809354782104, -0.9733660221099854, -1.0...",91116,/content/CodeFormer/align_hr_UTKface/91116.png,0.746011
2,/content/CodeFormer/align_hr_UTKface/70556.png,"[-0.6530567407608032, -0.26220884919166565, 1....",70556,/content/CodeFormer/align_hr_UTKface/70556.png,0.719257
3,/content/CodeFormer/align_hr_UTKface/72920.png,"[-2.296236753463745, 0.2565675973892212, 0.116...",72920,/content/CodeFormer/align_hr_UTKface/72920.png,0.194310
4,/content/CodeFormer/align_hr_UTKface/70545.png,"[-0.7272638082504272, 1.2198829650878906, 0.95...",70545,/content/CodeFormer/align_hr_UTKface/70545.png,0.516199
...,...,...,...,...,...
23950,/content/CodeFormer/align_hr_UTKface/86601.png,"[0.2633347511291504, 0.4482688009738922, 1.175...",86601,/content/CodeFormer/align_hr_UTKface/86601.png,0.175016
23951,/content/CodeFormer/align_hr_UTKface/87145.png,"[-2.198603630065918, 0.04760923981666565, 1.21...",87145,/content/CodeFormer/align_hr_UTKface/87145.png,0.502189
23952,/content/CodeFormer/align_hr_UTKface/89715.png,"[-0.8513012528419495, 0.602628767490387, 0.446...",89715,/content/CodeFormer/align_hr_UTKface/89715.png,0.903959
23953,/content/CodeFormer/align_hr_UTKface/89977.png,"[-3.269528388977051, 0.27106744050979614, 2.29...",89977,/content/CodeFormer/align_hr_UTKface/89977.png,0.489398


In [28]:
results_data.drop(['Path_x','Path_y'] , axis=1, inplace = True)

In [29]:
results_data

,embedding,CI_id,Quality
0,"[-1.210444688796997, -0.4962098002433777, 1.04...",91557,0.796164
1,"[0.9080809354782104, -0.9733660221099854, -1.0...",91116,0.746011
2,"[-0.6530567407608032, -0.26220884919166565, 1....",70556,0.719257
3,"[-2.296236753463745, 0.2565675973892212, 0.116...",72920,0.194310
4,"[-0.7272638082504272, 1.2198829650878906, 0.95...",70545,0.516199
...,...,...,...
23950,"[0.2633347511291504, 0.4482688009738922, 1.175...",86601,0.175016
23951,"[-2.198603630065918, 0.04760923981666565, 1.21...",87145,0.502189
23952,"[-0.8513012528419495, 0.602628767490387, 0.446...",89715,0.903959
23953,"[-3.269528388977051, 0.27106744050979614, 2.29...",89977,0.489398


# Datos para la base de datos
Revisar estos datos si cambiamos de database. o hacer las queries para obtenerlos. 

In [30]:
detector_id = 2 # mtcnn-serfiq
embeddingModel_id = 6 #El nuevo
qualityModel_id = 1 #ser_fiq

# Creamos/Actualizamos FaceImage

In [31]:
#buscamos la faceimage y la creamos si no existe
limit = 300
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    #cI = session.query(CroppedImage).join(Image).filter(Image.path == dato.Path,CroppedImage.detector_id==detector_id).one_or_none()
    cI = session.query(CroppedImage).filter(CroppedImage.croppedImage_id == dato.CI_id,CroppedImage.detector_id==detector_id).one_or_none()

    if cI:
        FI = session.query(FaceImage).filter(FaceImage.croppedImages == cI,FaceImage.embeddingModel_id == embeddingModel_id).one_or_none()
        if FI:
            FI.embeddings=dato.embedding

        else:
            new_face_image = FaceImage(croppedImage_id=cI.croppedImage_id,
            embeddingModel_id=embeddingModel_id,
            embeddings=dato.embedding,
            confusion_score=None)
            session.add(new_face_image)
        i-=1
        #session.commit()        

    else:
        print (f'{dato.Path} no encontrado')

    if not i:
        i=limit
        session.commit()

session.commit()
session.close()



23955it [00:39, 601.81it/s]


# Creamos / actualizamos QualityImage

In [32]:
#buscamos la qualityimage y la creamos si no existe
limit = 100
i = limit
for  index, dato in tqdm(results_data.iterrows()):
    FI = session.query(FaceImage).filter(FaceImage.embeddingModel_id == embeddingModel_id) \
                .join(CroppedImage).filter(CroppedImage.croppedImage_id == dato.CI_id).one_or_none()
#                .join(Image).filter(Image.path == dato.Path)

    if FI:
        QI = session.query(QualityImage).filter(QualityImage.faceImages == FI, QualityImage.qualityModel_id == qualityModel_id).one_or_none()
        if QI:
            QI.quality=dato.Quality

        else:
            new_quality_image = QualityImage(faceImage_id = FI.faceImage_id,
            qualityModel_id = qualityModel_id,
            quality=dato.Quality)
            session.add(new_quality_image)
        i-=1
        #session.commit()        

    else:
        print (dato.Path + 'no encontrado')

    if not i:
        i=limit
        session.commit()

session.commit()
session.close()





23955it [00:40, 586.94it/s]
